In [8]:
# Import Necessary Dependencies
import pandas as pd
import datetime as dt
import json
import pymongo
import numpy as np
import io
from sqlalchemy import create_engine

In [9]:
# Create paths for the csv files

PolluPath = "Resources/pollution_us_2000_2016.csv"
PopPath = "Resources/population_data.csv"

In [10]:
# Create data frame from CSV file variable(path) using pandas .read_csv() method
PolluDf = pd.read_csv(PolluPath)

# Print out first 10 rows for inspection
PolluDf.head()

,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.2,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.2,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.2,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.2,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.6,23,NaN


In [11]:
PolluDf['Date Local'] = pd.to_datetime(PolluDf['Date Local'], format= '%m/%d/%Y')
PolluDf['Year'] = PolluDf['Date Local'].dt.strftime('%Y')
PolluDf

ValueError: time data '2000-01-01' does not match format '%m/%d/%Y' (match)

In [ ]:
# Dropping unnecessary columns
# Only keep columns pertaininig to the scope of the project

cutPolluDf = PolluDf.drop(['Unnamed: 0', 'State Code','County Code','Site Num',
                           'Address', 'County', 'NO2 Units', 'O3 Units', 'SO2 Units',
                           'CO Units', 'NO2 1st Max Value', 'NO2 1st Max Hour', 'O3 1st Max Value',
                           'O3 1st Max Hour', 'SO2 1st Max Value', 'SO2 1st Max Hour', 'CO 1st Max Value',
                           'CO 1st Max Hour', 'State'], axis=1)
cutPolluDf

In [12]:
# Checking values in Race and Ethnicity to identify possible invalid/missing values 
#noPolluDf = cutPolluDf[cutPolluDf['NO2 AQI'] != 'NaN']
#oPolluDf = noPolluDf[noPolluDf['O3 AQI'] != 'NaN']
#soPolluDf = oPolluDf[oPolluDf['SO2 AQI'] != 'NaN']
#coPolluDf = soPolluDf[soPolluDf['CO AQI'] != 'NaN']
#coPolluDf

In [13]:
# Removing Pollution Data for all cities to limit the scope due to time constraints
SpecCityDf = cutPolluDf[(cutPolluDf['City'] == 'New York') |
                     (cutPolluDf['City'] == 'Los Angeles')]
SpecCityDf

NameError: name 'cutPolluDf' is not defined

In [164]:
# Dropping Incomplete Data Rows
CompleteCityDf = SpecCityDf.dropna()
CompleteCityDf

,City,Date Local,NO2 Mean,NO2 AQI,O3 Mean,O3 AQI,SO2 Mean,SO2 AQI,CO Mean,CO AQI,Year
12131,Los Angeles,2000-03-01,33.521739,45,0.010375,23,0.000000,0.0,1.004167,18.0,2000
12135,Los Angeles,2000-03-02,44.173913,52,0.009375,21,0.173913,1.0,1.537500,25.0,2000
12139,Los Angeles,2000-03-03,32.608696,40,0.012542,21,0.391304,4.0,0.729167,11.0,2000
12143,Los Angeles,2000-03-04,31.347826,40,0.016917,25,0.260870,4.0,0.712500,11.0,2000
12147,Los Angeles,2000-03-05,16.956522,40,0.021875,27,0.086957,3.0,0.404167,9.0,2000
...,...,...,...,...,...,...,...,...,...,...,...
1038663,New York,2010-12-27,10.500000,15,0.031500,30,3.437500,9.0,0.270833,3.0,2010
1038667,New York,2010-12-28,29.000000,47,0.013056,21,7.009524,24.0,0.308333,7.0,2010
1038671,New York,2010-12-29,25.125000,37,0.009292,17,5.375000,16.0,0.370833,7.0,2010
1038675,New York,2010-12-30,51.125000,95,0.002042,5,15.579167,40.0,0.858333,15.0,2010


In [165]:
# Make DataFrame Specific to Los Angeles
LaDf = CompleteCityDf[CompleteCityDf['City'] == 'Los Angeles']
LaDf.reset_index(drop=True, inplace=True)
LaDf

,City,Date Local,NO2 Mean,NO2 AQI,O3 Mean,O3 AQI,SO2 Mean,SO2 AQI,CO Mean,CO AQI,Year
0,Los Angeles,2000-03-01,33.521739,45,0.010375,23,0.000000,0.0,1.004167,18.0,2000
1,Los Angeles,2000-03-02,44.173913,52,0.009375,21,0.173913,1.0,1.537500,25.0,2000
2,Los Angeles,2000-03-03,32.608696,40,0.012542,21,0.391304,4.0,0.729167,11.0,2000
3,Los Angeles,2000-03-04,31.347826,40,0.016917,25,0.260870,4.0,0.712500,11.0,2000
4,Los Angeles,2000-03-05,16.956522,40,0.021875,27,0.086957,3.0,0.404167,9.0,2000
...,...,...,...,...,...,...,...,...,...,...,...
6563,Los Angeles,2010-12-27,25.865217,41,0.014625,25,0.469565,3.0,0.420833,7.0,2010
6564,Los Angeles,2010-12-28,28.563636,38,0.012208,19,0.368182,1.0,0.512500,8.0,2010
6565,Los Angeles,2010-12-29,15.700000,34,0.029042,34,0.018182,0.0,0.254167,6.0,2010
6566,Los Angeles,2010-12-30,5.147826,11,0.037542,35,0.000000,0.0,0.000000,0.0,2010


In [166]:
# Group DataFrame By Year
LaAnnual = LaDf.groupby('Year')
LaAnnualAvg = LaAnnual.mean()
LaAnnualAvg

,NO2 Mean,NO2 AQI,O3 Mean,O3 AQI,SO2 Mean,SO2 AQI,CO Mean,CO AQI
Year,,,,,,,,
2000,40.105582,60.385621,0.017689,32.722222,1.346163,7.140523,1.071678,18.503268
2001,37.947524,57.236307,0.018871,31.791862,2.495193,7.502347,1.150055,19.586854
2002,32.748161,49.872255,0.021647,33.708583,2.527785,6.650699,1.051268,18.159681
2003,33.625855,51.745665,0.018656,34.182081,2.058794,5.520231,0.963229,16.190751
2004,24.861311,41.173380,0.028356,41.443082,2.618151,6.973730,0.593673,10.663748
2005,20.047939,35.726768,0.023358,33.472954,3.736045,8.119279,0.437927,8.219140
2006,21.986027,37.374121,0.022269,32.140647,1.949943,6.053446,0.387275,7.566807
2007,21.490240,36.435860,0.023823,33.762391,1.809471,5.074344,0.386885,7.113703
2008,20.404282,35.327536,0.023782,34.159420,0.998528,3.453623,0.379771,6.769565


In [167]:
# Make DataFrame Specific to New York City
NyDf = CompleteCityDf[CompleteCityDf['City'] == 'New York']
NyDf.reset_index(drop=True, inplace=True)
NyDf

,City,Date Local,NO2 Mean,NO2 AQI,O3 Mean,O3 AQI,SO2 Mean,SO2 AQI,CO Mean,CO AQI,Year
0,New York,2000-05-05,42.500000,57,0.018417,31,8.800000,21.0,0.787500,13.0,2000
1,New York,2000-05-06,42.875000,81,0.034333,67,9.500000,37.0,1.195833,24.0,2000
2,New York,2000-05-07,29.000000,53,0.036583,80,7.041667,30.0,0.837500,15.0,2000
3,New York,2000-05-08,49.958333,86,0.027750,48,12.045455,37.0,0.916667,14.0,2000
4,New York,2000-05-09,36.619048,52,0.042125,71,9.666667,27.0,0.795833,14.0,2000
...,...,...,...,...,...,...,...,...,...,...,...
7970,New York,2010-12-27,10.500000,15,0.031500,30,3.437500,9.0,0.270833,3.0,2010
7971,New York,2010-12-28,29.000000,47,0.013056,21,7.009524,24.0,0.308333,7.0,2010
7972,New York,2010-12-29,25.125000,37,0.009292,17,5.375000,16.0,0.370833,7.0,2010
7973,New York,2010-12-30,51.125000,95,0.002042,5,15.579167,40.0,0.858333,15.0,2010


In [168]:
# Group DataFrame By Year
NyAnnual = NyDf.groupby('Year')
NyAnnualAvg = NyAnnual.mean()
NyAnnualAvg

,NO2 Mean,NO2 AQI,O3 Mean,O3 AQI,SO2 Mean,SO2 AQI,CO Mean,CO AQI
Year,,,,,,,,
2000,26.704985,41.027350,0.017815,27.082051,7.503272,22.494017,0.624132,10.152137
2001,28.523752,42.673028,0.017988,27.466921,8.583215,25.305344,0.618667,9.876590
2002,27.658741,41.456166,0.018097,30.288262,7.529247,23.401189,0.614898,9.444279
2003,26.741023,40.048529,0.017407,25.607353,8.020756,24.144118,0.700423,10.492647
2004,24.565002,38.018492,0.017231,25.251778,7.113480,22.334282,0.581190,8.611664
2005,25.699637,39.595129,0.019202,28.936073,7.425842,22.544901,0.555762,8.083714
2006,24.220642,38.096774,0.019435,27.617111,5.961814,19.695652,0.479111,7.228612
2007,23.271936,36.919323,0.019392,27.395418,5.597353,16.797809,0.509211,7.454183
2008,23.104233,37.113665,0.019759,27.486590,5.237649,15.242656,0.497514,7.358876


In [169]:
# POPULATION DATA IMPORTED FROM US CENSUS BUREAU

In [170]:
# Create data frame from CSV file variable(path) using pandas .read_csv() method
PopDf = pd.read_csv(PopPath, encoding = "ISO-8859-1", low_memory=False)
PopDf

,NAME,STNAME,ESTIMATESBASE2000,POPESTIMATE2000,POPESTIMATE2001,POPESTIMATE2002,POPESTIMATE2003,POPESTIMATE2004,POPESTIMATE2005,POPESTIMATE2006,POPESTIMATE2007,POPESTIMATE2008,POPESTIMATE2009,CENSUS2010POP,POPESTIMATE2010
0,California,California,33871653,33987977,34479458,34871843,35253159,35574576,35827943,36021202,36250311,36604337,36961229,37253956,37349363
1,Adelanto city,California,18135,18472,20039,21396,22921,24565,26138,27539,28728,29776,30801,31765,31874
2,Agoura Hills city,California,20466,20488,20600,20694,20748,20724,20631,20448,20289,20286,20320,20330,20355
3,Alameda city,California,72258,72524,73332,72785,72216,71623,71247,71211,71590,72481,73364,73812,73993
4,Albany city,California,16474,16579,16942,16984,17021,17049,17124,17270,17522,17909,18300,18539,18585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4559,Balance of Jerusalem town,New York,4414,4472,4457,4415,4407,4439,4466,4440,4437,4447,4402,4393,4392
4560,Balance of Milo town,New York,2399,2399,2397,2396,2422,2422,2426,2411,2434,2442,2438,2442,2441
4561,Balance of Potter town,New York,1398,1398,1394,1392,1404,1402,1402,1391,1403,1405,1401,1401,1400
4562,Balance of Starkey town,New York,1757,1771,1775,1773,1790,1803,1816,1812,1831,1843,1842,1848,1847


In [171]:
# Find Data Pertaining to the cities of Interest
# Los Angeles Population Data
LaPopRow = PopDf[PopDf['NAME'] == 'Los Angeles city']
LaPopRow

,NAME,STNAME,ESTIMATESBASE2000,POPESTIMATE2000,POPESTIMATE2001,POPESTIMATE2002,POPESTIMATE2003,POPESTIMATE2004,POPESTIMATE2005,POPESTIMATE2006,POPESTIMATE2007,POPESTIMATE2008,POPESTIMATE2009,CENSUS2010POP,POPESTIMATE2010
239,Los Angeles city,California,3694257,3701062,3733328,3762432,3784279,3792650,3788197,3767960,3751872,3763566,3781938,3792621,3797162
664,Los Angeles city,California,3694257,3701062,3733328,3762432,3784279,3792650,3788197,3767960,3751872,3763566,3781938,3792621,3797162


In [172]:
# New York Population Data
NyPopRow = PopDf[PopDf['NAME'] == 'New York city']
NyPopRow

,NAME,STNAME,ESTIMATESBASE2000,POPESTIMATE2000,POPESTIMATE2001,POPESTIMATE2002,POPESTIMATE2003,POPESTIMATE2004,POPESTIMATE2005,POPESTIMATE2006,POPESTIMATE2007,POPESTIMATE2008,POPESTIMATE2009,CENSUS2010POP,POPESTIMATE2010
1454,New York city,New York,8009185,8017608,8059813,8072000,8068073,8043366,8013368,7993906,8013775,8068195,8131574,8175133,8184899


In [173]:
# Now transform the rows of desired data into data frames that can be joined with Pollution Data
# Note: This is not the only way to accomplish this goal.
# Alternatively, the headers could have been renamed to simply the name of the years in a string form then the table could be pivoted then merged. 

# Los Angeles Population DataFrame
LaPopDf = pd.DataFrame({"Year": ["2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010"],
                       "Population": [3701062, 3733328, 3762432, 3784279, 3792650, 3788197, 3767960, 3751872, 3763566, 3781938, 3792661]})
LaPopDf.set_index('Year', inplace=True)
LaPopDf

,Population
Year,
2000,3701062
2001,3733328
2002,3762432
2003,3784279
2004,3792650
2005,3788197
2006,3767960
2007,3751872
2008,3763566


In [174]:
# New York City Population DataFrame to be joined
NyPopDf = pd.DataFrame({"Year": ["2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010"],
                       "Population": [8017608, 8059813, 8072000, 8068073, 8043366, 8013368, 7993906, 8013775, 8068195, 8131574, 8175133]})
#NyPopDf.set_index('Year', inplace=True)
NyPopDf

,Population
Year,
2000,8017608
2001,8059813
2002,8072000
2003,8068073
2004,8043366
2005,8013368
2006,7993906
2007,8013775
2008,8068195


In [175]:
# Joining these tables for comparison can be done very many different ways
# Final Los Angeles DataFrame to be Loaded into database

FinalLaDf = pd.concat([LaAnnualAvg, LaPopDf], axis=1, join="outer")
FinalLaDf

# Exporting LA Data Frame to a csv file to be imported into database
FinalLaDf.to_csv("../output_data/FinalLaDf.csv", index=True, header=True)

,NO2 Mean,NO2 AQI,O3 Mean,O3 AQI,SO2 Mean,SO2 AQI,CO Mean,CO AQI,Population
Year,,,,,,,,,
2000,40.105582,60.385621,0.017689,32.722222,1.346163,7.140523,1.071678,18.503268,3701062
2001,37.947524,57.236307,0.018871,31.791862,2.495193,7.502347,1.150055,19.586854,3733328
2002,32.748161,49.872255,0.021647,33.708583,2.527785,6.650699,1.051268,18.159681,3762432
2003,33.625855,51.745665,0.018656,34.182081,2.058794,5.520231,0.963229,16.190751,3784279
2004,24.861311,41.173380,0.028356,41.443082,2.618151,6.973730,0.593673,10.663748,3792650
2005,20.047939,35.726768,0.023358,33.472954,3.736045,8.119279,0.437927,8.219140,3788197
2006,21.986027,37.374121,0.022269,32.140647,1.949943,6.053446,0.387275,7.566807,3767960
2007,21.490240,36.435860,0.023823,33.762391,1.809471,5.074344,0.386885,7.113703,3751872
2008,20.404282,35.327536,0.023782,34.159420,0.998528,3.453623,0.379771,6.769565,3763566


In [176]:
# Final New York DataFrame to be Loaded into database

FinalNyDf = pd.concat([NyAnnualAvg, NyPopDf], axis=1, join="outer")
FinalNyDf

# Exporting LA Data Frame to a csv file to be imported into database
FinalNyDf.to_csv("../output_data/FinalNyDf.csv", index=True, header=True)

,NO2 Mean,NO2 AQI,O3 Mean,O3 AQI,SO2 Mean,SO2 AQI,CO Mean,CO AQI,Population
Year,,,,,,,,,
2000,26.704985,41.027350,0.017815,27.082051,7.503272,22.494017,0.624132,10.152137,8017608
2001,28.523752,42.673028,0.017988,27.466921,8.583215,25.305344,0.618667,9.876590,8059813
2002,27.658741,41.456166,0.018097,30.288262,7.529247,23.401189,0.614898,9.444279,8072000
2003,26.741023,40.048529,0.017407,25.607353,8.020756,24.144118,0.700423,10.492647,8068073
2004,24.565002,38.018492,0.017231,25.251778,7.113480,22.334282,0.581190,8.611664,8043366
2005,25.699637,39.595129,0.019202,28.936073,7.425842,22.544901,0.555762,8.083714,8013368
2006,24.220642,38.096774,0.019435,27.617111,5.961814,19.695652,0.479111,7.228612,7993906
2007,23.271936,36.919323,0.019392,27.395418,5.597353,16.797809,0.509211,7.454183,8013775
2008,23.104233,37.113665,0.019759,27.486590,5.237649,15.242656,0.497514,7.358876,8068195
